In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

df = pd.read_csv("../src/Flux_HistoPrice_example.csv")

## Explore the data

In [52]:
df.head()

,vnid,vintage,PriceHistory
0,1000001,2013,"[{202107,38}]"
1,1000232,2008,"[{201212,18},{201301,18},{201303,18},{201306,1..."
2,1000232,2007,"[{201203,18},{201206,19},{201209,18},{201303,2..."
3,1000245,2005,"[{201201,81},{201203,81},{201206,81},{201209,8..."
4,1000287,2010,"[{201506,19},{201612,41},{201701,40},{201703,4..."


In [53]:
df.columns

Index(['vnid', 'vintage', 'PriceHistory'], dtype='object')

In [54]:
df["vintage"].describe()

count      39.000000
mean     2009.743590
std         4.540565
min      1990.000000
25%      2008.000000
50%      2010.000000
75%      2012.000000
max      2019.000000
Name: vintage, dtype: float64

### Check if we have duplicates or null values

In [55]:
df.duplicated().sum()

1

In [56]:
duplicates = df[df.duplicated(keep=False)]
print(duplicates)

       vnid  vintage                                       PriceHistory
9   1000287     2011  [{201812,45},{201901,43},{201903,44},{201906,4...
10  1000287     2011  [{201812,45},{201901,43},{201903,44},{201906,4...


🔍 - Here, we have the same line. So I'm going to delete this duplicate.

In [57]:
df = df.drop_duplicates()

In [58]:
df.head(12)

,vnid,vintage,PriceHistory
0,1000001,2013,"[{202107,38}]"
1,1000232,2008,"[{201212,18},{201301,18},{201303,18},{201306,1..."
2,1000232,2007,"[{201203,18},{201206,19},{201209,18},{201303,2..."
3,1000245,2005,"[{201201,81},{201203,81},{201206,81},{201209,8..."
4,1000287,2010,"[{201506,19},{201612,41},{201701,40},{201703,4..."
5,1000287,1990,"[{200912,79},{201012,71},{201101,70},{201206,1..."
6,1000287,2008,"[{201606,41},{201609,41},{201612,42},{201701,4..."
7,1000287,2012,"[{201501,12},{201503,12},{202003,45},{202006,4..."
8,1000287,2009,"[{201403,41},{201406,41},{201409,35},{201412,4..."
9,1000287,2011,"[{201812,45},{201901,43},{201903,44},{201906,4..."


In [59]:
df.isna().sum()

vnid            0
vintage         0
PriceHistory    0
dtype: int64

## Refactoring the 'Pricehistory' column

### First try - ❌

In [60]:
# df["Date"] = None
# df["Prix"] = None


# for index, row in df.iterrows():
#     price_history = row["PriceHistory"]
#     for pair in eval(price_history):
#         date, price = pair
#         df.at[index, "Date"] = date
#         df.at[index, "Prix"] = price

### Second try - ✅

In [61]:
def extract_dates_prices(row):
    price_history = row["PriceHistory"]
    dates = []
    prices = []
    matches = re.findall(r"{(\d+),(\d+)}", price_history)
    for match in matches:
        date, price = match
        dates.append(int(date))
        prices.append(int(price))
    return dates, prices

In [62]:
df[["date", "price"]] = df.apply(lambda row: pd.Series(extract_dates_prices(row)), axis=1)

In [63]:
df.head()

,vnid,vintage,PriceHistory,date,price
0,1000001,2013,"[{202107,38}]",[202107],[38]
1,1000232,2008,"[{201212,18},{201301,18},{201303,18},{201306,1...","[201212, 201301, 201303, 201306, 201309, 20131...","[18, 18, 18, 18, 17, 17, 27, 27, 23, 25, 25, 2..."
2,1000232,2007,"[{201203,18},{201206,19},{201209,18},{201303,2...","[201203, 201206, 201209, 201303, 201306, 20130...","[18, 19, 18, 26, 26, 24, 28, 28, 33, 30, 30, 2..."
3,1000245,2005,"[{201201,81},{201203,81},{201206,81},{201209,8...","[201201, 201203, 201206, 201209, 201303, 20130...","[81, 81, 81, 81, 81, 82, 82, 92, 92, 92, 94, 9..."
4,1000287,2010,"[{201506,19},{201612,41},{201701,40},{201703,4...","[201506, 201612, 201701, 201703, 201706, 20170...","[19, 41, 40, 40, 40, 39, 38, 38, 42, 43, 47, 4..."


In [64]:
df = df.drop(["PriceHistory"], axis=1)

In [65]:
df.head()

,vnid,vintage,date,price
0,1000001,2013,[202107],[38]
1,1000232,2008,"[201212, 201301, 201303, 201306, 201309, 20131...","[18, 18, 18, 18, 17, 17, 27, 27, 23, 25, 25, 2..."
2,1000232,2007,"[201203, 201206, 201209, 201303, 201306, 20130...","[18, 19, 18, 26, 26, 24, 28, 28, 33, 30, 30, 2..."
3,1000245,2005,"[201201, 201203, 201206, 201209, 201303, 20130...","[81, 81, 81, 81, 81, 82, 82, 92, 92, 92, 94, 9..."
4,1000287,2010,"[201506, 201612, 201701, 201703, 201706, 20170...","[19, 41, 40, 40, 40, 39, 38, 38, 42, 43, 47, 4..."


#### Know I want convert date to datetime format

In [66]:
from datetime import datetime

data = []

# Iterate through each row
for i, row in df.iterrows():
    for date, price in zip(row['date'], row['price']):
        
        date_str = str(date) + "01"
        date_dt = datetime.strptime(date_str, '%Y%m%d')
        date_dt = date_dt.replace(day=1)
        
        data.append({'vnid': row['vnid'], 'vintage': row['vintage'], 'date': date_dt, 'price': price})

# Create new dataframe
df_new = pd.DataFrame(data)

In [67]:
df_new.info() # Check for type and null values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1254 entries, 0 to 1253
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   vnid     1254 non-null   int64         
 1   vintage  1254 non-null   int64         
 2   date     1254 non-null   datetime64[ns]
 3   price    1254 non-null   int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 39.3 KB


In [68]:
# df_new['date'] = pd.to_datetime(df_new['date'], format='%Y%m')

In [69]:
df_new.head()

,vnid,vintage,date,price
0,1000001,2013,2021-07-01,38
1,1000232,2008,2012-12-01,18
2,1000232,2008,2013-01-01,18
3,1000232,2008,2013-03-01,18
4,1000232,2008,2013-06-01,18


# 📌 Codex - VNID and VINTAGE

In [70]:
vnid_list = df_new["vnid"].unique()

In [71]:
for vnid in vnid_list:
    vnid_data = df_new[df_new["vnid"] == vnid]
    vintage_list = vnid_data["vintage"].unique()
    
    print(f"vnid : {vnid}, vintage : {vintage_list}")

vnid : 1000001, vintage : [2013]
vnid : 1000232, vintage : [2008 2007]
vnid : 1000245, vintage : [2005]
vnid : 1000287, vintage : [2010 1990 2008 2012 2009 2011 2013 2007]
vnid : 1000359, vintage : [2016]
vnid : 1000391, vintage : [2010]
vnid : 1000535, vintage : [2019 2007 2004 2018]
vnid : 1000548, vintage : [2011]
vnid : 1000564, vintage : [2011 2014 2012 2007 2008 2009 2010]
vnid : 1000577, vintage : [2011 2010 2012 2013 2007 2008 2009]
vnid : 1000580, vintage : [2013 2009 2011 2007]
vnid : 1000593, vintage : [2010]
